# BERT 預訓練模型載入

In [1]:
#!pip install transformers

In [2]:
import transformers
transformers. __version__ 

'4.16.2'

In [3]:
from transformers import TFBertModel, BertTokenizer
import tensorflow as tf

# download bert-base-uncased model 
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [4]:
# load資料
import pandas as pd
import numpy as np

#測試先讀入其中一個資料就好
aud1_df = pd.read_csv("./Pre_data/move_window/AUD/AUD1.csv", index_col = 0)
aud2_df = pd.read_csv("./Pre_data/move_window/AUD/AUD2.csv", index_col = 0)
aud3_df = pd.read_csv("./Pre_data/move_window/AUD/AUD3.csv", index_col = 0)

In [5]:
aud1_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12099 entries, 0 to 12098
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           12099 non-null  object 
 1   word2vec_body  12099 non-null  object 
 2   tfidf_body     12099 non-null  object 
 3   bert_body      12099 non-null  object 
 4   source_title   12099 non-null  object 
 5   binary         12099 non-null  float64
 6   ternary_1      12099 non-null  float64
 7   ternary_2      12099 non-null  float64
dtypes: float64(3), object(5)
memory usage: 850.7+ KB


In [133]:
def text2averege_bertvec(text):
    wordvec = np.array([])
    text_vec = np.array([])
    word_count = 0
                
    for segment in eval(text):
        if(segment != "" and segment != " "):
            tokens = tokenizer.tokenize(segment)                #分割句字中的單字，看不懂得bert會補上萬用字元
            token_ids = tokenizer.convert_tokens_to_ids(tokens) #將單字轉換為bert對應的詞id
            token_ids = tf.convert_to_tensor(token_ids)         #將id轉為tensor表示(詞數,)
            token_ids = tf.reshape(token_ids, [1, -1 ])         #reshape(1,詞數)

            output = bert_model(token_ids)                      #輸入至bert模型中\
            segment_vec = output[0][0].numpy()                  #將tensor轉為numpy，比較好用> <
            word_count += segment_vec.shape[0]                  #統計字數
            wordvec = np.append(wordvec, segment_vec)           #將所有句子append到一個np.array。缺點會全部變成一維表示
        
    wordvec = wordvec.reshape(word_count,768)               #重塑1D->2D <字數，詞向量>
    
    text_vec = np.mean(wordvec,axis=0)                      #對col方向進行平均計算
    
    return text_vec.reshape(1,768)                          #重塑矩陣形狀為1一文本300文本向量(這是一個確保，以免形狀不對)

In [131]:
def text2_bertvec(text, max_word = 100):
    wordvec = np.array([])
    word_count = 0
                
    for segment in eval(text):
        if(segment != "" and segment != " "):
            tokens = tokenizer.tokenize(segment)                #分割句字中的單字，看不懂得bert會補上萬用字元
            token_ids = tokenizer.convert_tokens_to_ids(tokens) #將單字轉換為bert對應的詞id
            token_ids = tf.convert_to_tensor(token_ids)         #將id轉為tensor表示(詞數,)
            token_ids = tf.reshape(token_ids, [1, -1 ])         #reshape(1,詞數)

            output = bert_model(token_ids)                      #輸入至bert模型中\
            segment_vec = output[0][0].numpy()                  #將tensor轉為numpy，比較好用> <
            word_count += segment_vec.shape[0]                  #統計字數
            wordvec = np.append(wordvec, segment_vec)           #將所有句子append到一個np.array。缺點會全部變成一維表示
        
    wordvec = wordvec.reshape(word_count,768)               #重塑1D->2D <字數，詞向量>
    
    
    if word_count > max_word:                              
        wordvec = wordvec[0 : max_word]                     #字多就砍，保留前面
    
    
    elif word_count < max_word:                             #字少就加，添加0向量，補足我們需要的大小
        pad_num = max_word - word_count
        filler = np.zeros((pad_num, 768))
        wordvec =np.append(wordvec,filler,axis=0)           #這用法很重要ㄟ，很好用!!!
    
    return wordvec

In [115]:
#挑選YF的來源測試。
groupset = aud1_df.groupby("source_title") #分群
aud1_YF = groupset.get_group("Yahoo Finance") #取出YF
aud1_YF = aud1_YF.reset_index(drop=True) #重設索引

In [116]:
#設定變數
df_len = aud1_YF.shape[0] #新聞數
bertvec = np.array([]) #新聞向量儲存

#計算文本向量
for i in range(df_len):
    bertvec = np.append(bertvec, text2averege_bertvec(aud1_YF["bert_body"][i]))
bertvec = bertvec.reshape(df_len,768) #使每篇文本有各自的向量 

InvalidArgumentError: Exception encountered when calling layer "embeddings" (type TFBertEmbeddings).

Value for attr 'Tindices' of float is not in the list of allowed values: int32, int64
	; NodeDef: {{node ResourceGather}}; Op<name=ResourceGather; signature=resource:resource, indices:Tindices -> output:dtype; attr=batch_dims:int,default=0; attr=validate_indices:bool,default=true; attr=dtype:type; attr=Tindices:type,allowed=[DT_INT32, DT_INT64]; is_stateful=true> [Op:ResourceGather]

Call arguments received:
  • input_ids=tf.Tensor(shape=(1, 0), dtype=float32)
  • position_ids=None
  • token_type_ids=tf.Tensor(shape=(1, 0), dtype=int32)
  • inputs_embeds=None
  • past_key_values_length=0
  • training=False

In [118]:
len(bertvec)

11520

In [121]:
text = aud1_YF["bert_body"][15]

In [122]:
text

"['melbourne reuters australia post has successfully fieldtrialled the use of drones to deliver small packages clearing the way for test deliveries to customer homes later this year', 'the stateowned corporation said the drones will be used for the delivery of online shopping parcels and timesensitive items such as medication', 'we will put this innovative technology through its paces over the coming weeks and months to understand what it can deliver how far it can travel and ultimately how our customers could receive a parcel australia post managing director ahmed fahour said', 'postal services around the world are facing dramatic declines in their core letterdelivery business as customers turn to the internet for all forms of correspondence from billing to greeting cards', 'while australia post is the first australian company to get into the area of drone delivery in the united states online retailer amazon unveiled a drone delivery prototype in november joining competitors googl and

In [134]:
text2averege_bertvec(text)

array([[ 5.14824732e-02,  1.88410339e-01,  4.30045264e-01,
         1.72439997e-01,  2.18442846e-01, -1.21281643e-01,
        -3.88794386e-01,  4.34365995e-01,  2.52648963e-02,
        -3.60528971e-01,  3.13426118e-03, -5.20747897e-01,
         2.87111282e-02,  3.39647215e-01, -3.61451245e-02,
         4.70918997e-01, -1.45415441e-01,  1.84280125e-01,
        -2.88059543e-01,  1.74887497e-01,  5.57171981e-02,
        -1.85740338e-01,  2.48345919e-01,  4.83319071e-01,
         3.95006776e-01, -3.14991084e-01,  1.66375492e-01,
        -1.57143173e-02, -3.59626700e-01, -6.32379781e-02,
         4.26675032e-01, -2.09261803e-01, -1.47184285e-01,
        -1.10457317e-01,  1.23040926e-01, -9.21108983e-02,
         1.55168864e-02,  1.15635396e-01, -2.72290079e-01,
        -1.30408452e-01, -6.56710400e-01, -2.88112444e-01,
         1.28554696e-01, -1.48023918e-01, -4.44041090e-01,
        -6.40368800e-02,  1.06219896e-02, -2.00929059e-01,
         5.52311321e-02,  9.40183956e-02, -4.61942272e-0

In [128]:
wordvec = np.array([])
text_vec = np.array([])
word_count = 0
                
for segment in eval(text):
    
        tokens = tokenizer.tokenize(segment)                #分割句字中的單字，看不懂得bert會補上萬用字元
        print(tokens)
        token_ids = tokenizer.convert_tokens_to_ids(tokens) #將單字轉換為bert對應的詞id
        print(token_ids)
        token_ids = tf.convert_to_tensor(token_ids)         #將id轉為tensor表示(詞數,)
        print(token_ids)
        token_ids = tf.reshape(token_ids, [1, -1 ])         #reshape(1,詞數)
        print(token_ids)

        output = bert_model(token_ids)                      #輸入至bert模型中\
        segment_vec = output[0][0].numpy()                  #將tensor轉為numpy，比較好用> <
        word_count += segment_vec.shape[0]                  #統計字數
        wordvec = np.append(wordvec, segment_vec)           #將所有句子append到一個np.array。缺點會全部變成一維表示

['melbourne', 'reuters', 'australia', 'post', 'has', 'successfully', 'field', '##tri', '##alle', '##d', 'the', 'use', 'of', 'drones', 'to', 'deliver', 'small', 'packages', 'clearing', 'the', 'way', 'for', 'test', 'deliveries', 'to', 'customer', 'homes', 'later', 'this', 'year']
[4940, 26665, 2660, 2695, 2038, 5147, 2492, 18886, 24164, 2094, 1996, 2224, 1997, 24633, 2000, 8116, 2235, 14555, 8430, 1996, 2126, 2005, 3231, 23534, 2000, 8013, 5014, 2101, 2023, 2095]
tf.Tensor(
[ 4940 26665  2660  2695  2038  5147  2492 18886 24164  2094  1996  2224
  1997 24633  2000  8116  2235 14555  8430  1996  2126  2005  3231 23534
  2000  8013  5014  2101  2023  2095], shape=(30,), dtype=int32)
tf.Tensor(
[[ 4940 26665  2660  2695  2038  5147  2492 18886 24164  2094  1996  2224
   1997 24633  2000  8116  2235 14555  8430  1996  2126  2005  3231 23534
   2000  8013  5014  2101  2023  2095]], shape=(1, 30), dtype=int32)
['the', 'state', '##own', '##ed', 'corporation', 'said', 'the', 'drones', 'will', 'b

In [125]:
text

"['melbourne reuters australia post has successfully fieldtrialled the use of drones to deliver small packages clearing the way for test deliveries to customer homes later this year', 'the stateowned corporation said the drones will be used for the delivery of online shopping parcels and timesensitive items such as medication', 'we will put this innovative technology through its paces over the coming weeks and months to understand what it can deliver how far it can travel and ultimately how our customers could receive a parcel australia post managing director ahmed fahour said', 'postal services around the world are facing dramatic declines in their core letterdelivery business as customers turn to the internet for all forms of correspondence from billing to greeting cards', 'while australia post is the first australian company to get into the area of drone delivery in the united states online retailer amazon unveiled a drone delivery prototype in november joining competitors googl and